## Extract Phase.
#### Donwloads from datasource tar.gz files containing geographical coordinates, saves and extracts on a destination folder.
The `extract_from_source()` is a python module that receives a url containing a `tar.gz` datasource, a destination directory and a flag to exctract the content.
The output is files extracted at destination directory.

In [1]:
from extract_source import extract_from_source
 
source = "https://s3.amazonaws.com/dev.etl.python/datasets/data_points.tar.gz"
destination_path = "/app/data_from_source"
extract = True

extract_from_source(source, destination_path, extract)


File downloaded from https://s3.amazonaws.com/dev.etl.python/datasets/data_points.tar.gz to /app/data_from_source/data_points.tar.gz.
Files extracted at /app/data_from_source directory.


# Transform Phase

#### 1 - Reads a directory containing coordinates/points data files.

In [2]:
from transform_csv import get_data_files

files_path = "/app/data_from_source"
files = get_data_files("/app/data_from_source")


Files found data_points_20180103.txt data_points_20180102.txt data_points_20180101.txt


#### 2 - Prepares for cleaning, transforming file content to `List[List[str()]]`

In [3]:
from transform_csv import wrangle_points_to_list

data_files = []
for file in files:
    data_files.append(f"{files_path}/{file}")
points_list = wrangle_points_to_list(data_files)
print(points_list)


[['Latitude: 30°02′28″S   -30.04108716', 'Longitude: 51°13′56″W   -51.23222954', 'Distance: 1.5753 km  Bearing: 242.95°'], ['Latitude: 30°03′59″S   -30.06668175', 'Longitude: 51°11′10″W   -51.18600466', 'Distance: 4.6886 km  Bearing: 139.468°'], ['Latitude: 30°00′52″S   -30.01448479', 'Longitude: 51°12′43″W   -51.21205259', 'Distance: 2.3066 km  Bearing: 13.536°'], ['Latitude: 30°00′06″S   -30.00155488', 'Longitude: 51°13′57″W   -51.23262034', 'Distance: 3.9527 km  Bearing: 338.617°'], ['Latitude: 30°01′40″S   -30.02779224', 'Longitude: 51°10′16″W   -51.17100925', 'Distance: 4.5563 km  Bearing: 80.379°'], ['Latitude: 29°59′37″S   -29.99357858', 'Longitude: 51°14′08″W   -51.23541934', 'Distance: 4.8777 km  Bearing: 339.466°'], ['Latitude: 30°00′59″S   -30.01648922', 'Longitude: 51°14′58″W   -51.24954632', 'Distance: 3.6754 km  Bearing: 303.324°'], ['Latitude: 30°03′31″S   -30.05849852', 'Longitude: 51°11′12″W   -51.18672493', 'Distance: 3.9885 km  Bearing: 131.701°'], ['Latitude: 30°02′

#### 3 - Apply data detection using regexp and store in a`list[dict{}]`

In [4]:
from transform_csv import convert_data_coordinates

detected_points = convert_data_coordinates(points_list)

print(detected_points)


[{'latitude': -30.04108716, 'longitude': -51.23222954, 'distance_km': 1.5753, 'bearing_degrees': 242.95}, {'latitude': -30.06668175, 'longitude': -51.18600466, 'distance_km': 4.6886, 'bearing_degrees': 139.468}, {'latitude': -30.01448479, 'longitude': -51.21205259, 'distance_km': 2.3066, 'bearing_degrees': 13.536}, {'latitude': -30.00155488, 'longitude': -51.23262034, 'distance_km': 3.9527, 'bearing_degrees': 338.617}, {'latitude': -30.02779224, 'longitude': -51.17100925, 'distance_km': 4.5563, 'bearing_degrees': 80.379}, {'latitude': -29.99357858, 'longitude': -51.23541934, 'distance_km': 4.8777, 'bearing_degrees': 339.466}, {'latitude': -30.01648922, 'longitude': -51.24954632, 'distance_km': 3.6754, 'bearing_degrees': 303.324}, {'latitude': -30.05849852, 'longitude': -51.18672493, 'distance_km': 3.9885, 'bearing_degrees': 131.701}, {'latitude': -30.04627163, 'longitude': -51.24721484, 'distance_km': 3.1258, 'bearing_degrees': 245.559}, {'latitude': -30.05341701, 'longitude': -51.2094

#### 4 - Remove duplicated dictionaries inside `detected_points` list.

In [5]:
from transform_csv import  remove_duplicates

print(f"Before deduplication {len(detected_points)}")

deduplicated_points = remove_duplicates(detected_points)

print(f"After deduplication {len(deduplicated_points)}")
print(deduplicated_points)


Before deduplication 2991
After deduplication 2991
[{'latitude': -30.04108716, 'longitude': -51.23222954, 'distance_km': 1.5753, 'bearing_degrees': 242.95}, {'latitude': -30.06668175, 'longitude': -51.18600466, 'distance_km': 4.6886, 'bearing_degrees': 139.468}, {'latitude': -30.01448479, 'longitude': -51.21205259, 'distance_km': 2.3066, 'bearing_degrees': 13.536}, {'latitude': -30.00155488, 'longitude': -51.23262034, 'distance_km': 3.9527, 'bearing_degrees': 338.617}, {'latitude': -30.02779224, 'longitude': -51.17100925, 'distance_km': 4.5563, 'bearing_degrees': 80.379}, {'latitude': -29.99357858, 'longitude': -51.23541934, 'distance_km': 4.8777, 'bearing_degrees': 339.466}, {'latitude': -30.01648922, 'longitude': -51.24954632, 'distance_km': 3.6754, 'bearing_degrees': 303.324}, {'latitude': -30.05849852, 'longitude': -51.18672493, 'distance_km': 3.9885, 'bearing_degrees': 131.701}, {'latitude': -30.04627163, 'longitude': -51.24721484, 'distance_km': 3.1258, 'bearing_degrees': 245.559

#### 5 - Converts the `List[dict{}]` of deduplicated points in a csv file and saves at the disk.

In [6]:
from transform_csv import write_points_to_csv

path_to_csv = "/app/normalized_data/data.csv"

write_points_to_csv(deduplicated_points, path_to_csv)

with open(path_to_csv, "r") as csv_file:
    [print(line) for line in csv_file]


CSV `data.csv` file saved at `/app/normalized_data`
latitude,longitude,distance_km,bearing_degrees

-30.04108716,-51.23222954,1.5753,242.95

-30.06668175,-51.18600466,4.6886,139.468

-30.01448479,-51.21205259,2.3066,13.536

-30.00155488,-51.23262034,3.9527,338.617

-30.02779224,-51.17100925,4.5563,80.379

-29.99357858,-51.23541934,4.8777,339.466

-30.01648922,-51.24954632,3.6754,303.324

-30.05849852,-51.18672493,3.9885,131.701

-30.04627163,-51.24721484,3.1258,245.559

-30.05341701,-51.20947473,2.2315,159.325

-30.00870545,-51.22392078,2.9477,348.193

-30.05029304,-51.26216174,4.6248,247.885

-30.07790316,-51.2036786,4.9959,164.376

-30.05968178,-51.1958636,3.4866,143.004

-30.01243072,-51.25827784,4.6268,302.27

-30.03972704,-51.19005805,2.717,102.01

-30.06070468,-51.17764669,4.8208,126.966

-30.04533078,-51.2212426,1.2374,196.193

-29.99904125,-51.21698046,3.9608,0.945

-30.05241274,-51.18524664,3.6937,122.35

-30.05119316,-51.23154781,2.2749,216.001

-30.05804203,-51.20906409,2.73

-29.99838659,-51.20285681,4.2776,19.47

-29.99218505,-51.23068368,4.8867,345.121

-30.06657041,-51.18420785,4.7936,137.801

-30.05119601,-51.25191386,3.7771,240.826

-30.06909688,-51.21168502,3.8746,171.466

-30.02906545,-51.23950913,2.1938,286.434

-29.99615389,-51.19218421,4.9346,29.82

-30.06972153,-51.20102131,4.2172,157.683

-30.01471838,-51.24483085,3.4294,310.261

-30.04789366,-51.19737292,2.4466,127.034

-30.00232262,-51.20096454,3.9384,24.097

-30.06646111,-51.20117983,3.8779,155.855

-30.0310675,-51.22815946,1.0867,291.489

-30.04510488,-51.21027054,1.3635,148.554

-29.99501607,-51.24106345,4.9509,332.91

-29.99872776,-51.1947129,4.5656,28.954

-30.00111927,-51.20138789,4.045,22.796

-30.05131808,-51.1854074,3.6167,120.851

-30.03367454,-51.17177676,4.4193,88.609

-30.03597666,-51.25175233,3.2863,267.412

-30.06604635,-51.23999611,4.1015,211.619

-29.99987283,-51.20095872,4.1889,22.583

-30.006791,-51.20604537,3.294,19.851

-30.01027192,-51.2159827,2.716,3.407

-30.01059807,-

-30.00425345,-51.21913349,3.3836,357.593

-30.04417838,-51.22543675,1.298,215.238

-30.02531929,-51.2461985,2.9376,290.675

-30.04726567,-51.18579077,3.3741,114.588

-30.05956427,-51.18155161,4.4459,128.572

-30.00633961,-51.19409358,3.8812,35.789

-30.00572583,-51.22480993,3.2897,347.913

-30.01614288,-51.21228745,2.1222,14.108

-30.04910573,-51.20852449,1.8329,151.33

-30.04110432,-51.17257192,4.4003,99.405

-30.04715509,-51.21316462,1.4569,162.725

-30.00648227,-51.24612937,4.1631,318.799

-30.0484934,-51.23805099,2.4954,231.886

-30.03184484,-51.26300193,4.3774,274.072

-30.03074101,-51.20914832,0.9275,62.071

-30.02466488,-51.19017055,2.8704,67.251

-30.07460458,-51.19784512,4.8364,156.776

-30.04569132,-51.19475398,2.5244,119.124

-30.02644543,-51.21549246,0.9358,12.879

-30.06826071,-51.22656554,3.8358,192.916

-30.07570533,-51.23296515,4.7986,197.88

-30.03286503,-51.21912561,0.2434,324.519

-30.0743015,-51.23994967,4.905,205.94

-30.01056824,-51.17604317,4.8202,56.257

-30.002

-30.03023274,-51.19797945,1.9575,75.479

-30.03180291,-51.24264482,2.4267,277.484

-30.06369814,-51.24927693,4.4393,223.284

-30.03646187,-51.17000633,4.5929,92.531

-30.02397132,-51.19537614,2.4523,61.045

-30.01408348,-51.18328946,4.0232,55.362

-30.05681279,-51.20258817,2.8607,149.526

-30.05623054,-51.25910312,4.6568,238.958

-30.01683332,-51.18427581,3.7763,58.361

-30.04454902,-51.19999435,2.0263,122.929

-30.05811293,-51.18201996,4.3111,127.268

-30.07337274,-51.24228927,4.9169,208.828

-30.04979864,-51.17091746,4.8056,110.541

-30.03485368,-51.17579986,4.0307,90.337

-30.02415946,-51.2229535,1.2731,336.388

-30.01466645,-51.21919765,2.2273,356.184

-30.05908983,-51.22932791,2.9417,202.451

-30.03894838,-51.22821897,1.1238,244.801

-30.02498594,-51.19644174,2.3084,62.263

-30.0330561,-51.25123708,3.2382,273.124

-30.01519644,-51.23960536,3.0245,315.663

-30.01664905,-51.19020366,3.3163,52.875

-30.03176919,-51.16829614,4.764,86.16

-30.02345119,-51.185796,3.3113,67.918

-30.0205


-30.03200208,-51.21124432,2.8398,301.326

-30.05145152,-51.17388897,0.6841,64.532

-30.00013013,-51.25023119,4.6101,113.929

-30.03394086,-51.24611964,4.9579,320.739

-30.00291729,-51.24047214,2.7417,271.635

-30.06653204,-51.18860918,4.1572,328.09

-30.07246971,-51.2348508,4.5165,141.748

-30.06879218,-51.24808444,4.5208,201.472

-30.05239586,-51.17580198,4.7963,217.635

-30.05443214,-51.17259794,4.4876,116.109

-30.00971999,-51.18576295,4.8647,116.907

-30.03677499,-51.21018258,4.1379,47.938

-30.07371338,-51.20410129,0.7578,108.202

-30.04769486,-51.25926818,4.537,163.285

-30.00780677,-51.1772876,4.2611,250.077

-30.03872221,-51.21655363,4.9018,52.491

-30.07605407,-51.21353696,0.4656,166.792

-29.99741826,-51.22718923,4.6226,175.077

-30.06605199,-51.18125966,4.2413,347.499

-30.00578164,-51.25386433,4.9479,134.917

-30.04615278,-51.2259926,4.7398,312.629

-30.03326925,-51.25611761,1.5105,212.088

-30.03315749,-51.17676907,3.7065,272.36

-30.05004668,-51.2188864,3.9408,87.601

-2


-30.02519576,-51.26171309,4.3706,283.907

-30.029964,-51.24266566,2.4637,282.199

-30.05508481,-51.23826596,3.0173,221.11

-30.03282018,-51.17690912,3.9291,87.046

-30.07488244,-51.23654659,4.8306,202.108

-30.02028484,-51.20780071,1.8582,30.723

-30.01552044,-51.2296043,2.4185,331.594

-30.03251607,-51.20545432,1.1988,78.6

-29.99553446,-51.20213845,4.6,18.966

-29.99800504,-51.24623375,4.9177,325.963

-30.05710886,-51.25779028,4.6012,237.104

-30.01026382,-51.21704561,2.7127,1.247

-30.04660808,-51.21053979,1.4966,152.744

-30.04404987,-51.1732447,4.4025,103.753

-29.99828199,-51.18727592,4.9922,35.891

-30.05260148,-51.19324739,3.0842,130.359

-30.07711343,-51.2222012,4.7436,185.289

-30.03380976,-51.22879742,1.0766,274.96

-30.05348368,-51.17394029,4.7019,116.472

-30.06161656,-51.23251074,3.3231,205.483

-30.04964102,-51.24953279,3.4928,241.472

-30.00447198,-51.23943018,3.9574,328.0

-30.01709159,-51.23857695,2.8055,314.102

-30.04950345,-51.26128664,4.514,248.516

-30.02123332,


-30.01349407,-51.20207148,4.1996,159.068

-30.00666514,-51.17521126,4.7165,60.087

-30.0256826,-51.19829065,3.6284,30.939

-30.03442528,-51.2579493,4.0059,284.403

-30.04264495,-51.26356533,4.4205,270.308

-30.01944113,-51.20545731,1.4736,127.136

-30.04268735,-51.20167921,2.2866,42.3

-30.06819727,-51.23649071,2.0218,243.744

-30.04559231,-51.22291981,3.7659,187.729

-30.02841606,-51.25043319,3.3824,248.897

-30.06744597,-51.1734405,4.314,80.766

-30.06533007,-51.24488393,4.4921,215.69

-30.02432516,-51.23821457,3.9451,210.104

-30.01888578,-51.20982661,1.3736,33.303

-30.02129792,-51.2122941,1.8276,16.42

-30.02433608,-51.17891759,4.0153,68.307

-30.02250717,-51.24530285,2.8986,293.3

-29.99993691,-51.24271871,2.7653,299.222

-30.04391399,-51.22537552,3.9316,349.101

-30.03767436,-51.24119428,2.4896,245.537

-30.07105246,-51.20012106,1.7219,101.281

-30.01955956,-51.20162131,4.3336,159.133

-30.04326351,-51.18924931,3.2094,58.482

-30.04460101,-51.24695316,2.9791,251.227

-30.012238

-30.06302793,-51.22376941,3.2111,190.556

-30.01952008,-51.20514812,2.0694,35.607

-30.03686196,-51.19299858,2.3872,95.929

-30.07872469,-51.22191476,4.9197,184.777

-30.04481263,-51.22652103,1.4166,217.041

-30.05607432,-51.19330627,3.3433,135.474

-30.06426831,-51.24233683,4.062,215.791

-30.07179638,-51.245152,4.9071,212.636

-30.05287994,-51.23301315,2.5096,216.088

-30.01077942,-51.20120767,3.0915,30.832

-30.02729918,-51.25371236,3.5667,283.238

-29.99314167,-51.20865134,4.6973,10.645

-30.00749021,-51.1952586,3.7118,35.539

-30.02475543,-51.2362352,2.1001,301.588

-30.02056663,-51.22589027,1.7553,333.151

-30.04463483,-51.18914007,2.9621,112.034

-30.03829058,-51.20466946,1.3147,107.957

-30.07360398,-51.19656867,4.7851,154.899

-30.04047134,-51.17075009,4.5629,98.174

-30.03579793,-51.25802842,3.8894,268.104

-30.02832311,-51.2155522,0.732,16.085

-30.02624934,-51.23694038,2.0785,296.7

-30.01235703,-51.20336983,2.8355,29.035

-30.00888819,-51.23186313,3.1749,334.474

-30.04523

-30.04435477,-51.23981357,3.5059,322.507

-30.05251535,-51.26623594,4.8004,256.989

-30.06766512,-51.25387566,4.0137,240.311

-30.00668866,-51.23029285,3.8687,198.323

-30.02658238,-51.21575118,3.1151,3.381

-30.06523906,-51.2418105,2.4927,291.085

-30.03317136,-51.25475152,4.9327,226.375

-30.06457811,-51.26830224,4.8794,271.915

-30.04781455,-51.2353584,3.7399,207.102

-30.05603536,-51.18126729,3.7977,112.693

-30.03294619,-51.19152666,3.4626,133.403

-30.02386726,-51.16832218,4.7544,87.732

-30.01986809,-51.18562751,3.3093,68.766

-30.00657367,-51.18678828,3.397,61.067



#### 6 - Reads data from CSV file, converts and returns a Dataframe containing the values.

In [7]:
from transform_db import Converter

# Gets the API_KEY form .env file
converter = Converter()
dataset_from_csv = converter.get_coordinates_from_csv_file(path_to_csv)

print(dataset_from_csv)

       latitude  longitude  distance_km  bearing_degrees
0    -30.041087 -51.232230       1.5753          242.950
1    -30.066682 -51.186005       4.6886          139.468
2    -30.014485 -51.212053       2.3066           13.536
3    -30.001555 -51.232620       3.9527          338.617
4    -30.027792 -51.171009       4.5563           80.379
5    -29.993579 -51.235419       4.8777          339.466
6    -30.016489 -51.249546       3.6754          303.324
7    -30.058499 -51.186725       3.9885          131.701
8    -30.046272 -51.247215       3.1258          245.559
9    -30.053417 -51.209475       2.2315          159.325
10   -30.008705 -51.223921       2.9477          348.193
11   -30.050293 -51.262162       4.6248          247.885
12   -30.077903 -51.203679       4.9959          164.376
13   -30.059682 -51.195864       3.4866          143.004
14   -30.012431 -51.258278       4.6268          302.270
15   -30.039727 -51.190058       2.7170          102.010
16   -30.060705 -51.177647     

#### 7 - Makes the API calls to retrieve data from latitude/longitude points and saves to database.

In [ ]:
converter.save_dataset_coordinates_to_database(dataset_from_csv)


Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '2351', 'street_name': 'Rua Coronel Aparício Borges', 'neighborhood': 'Coronel Aparicio Borges', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90680-570', 'latitude': -30.06668175, 'longitude': -51.18600466}
Address saved to database: {'street_number': '2136', 'street_name': 'Rua Voluntários da Pátria', 'neighborhood': 'Centro Histórico de Porto Alegre', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90030-001', 'latitude': -30.01448479, 'longitude': -51.21205259}
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '2098', 'street_name': 'Avenida Doutor Nilo Peçanha', 'neighborhood': 'Petrópolis', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90470-001', 'latitude': -30.02779224, 'longitude': -51.171

Address saved to database: {'street_number': '220', 'street_name': 'Rua Comendador Caminha', 'neighborhood': 'Moinhos de Vento', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90430-030', 'latitude': -30.02655075, 'longitude': -51.20207391}
Address saved to database: {'street_number': '50', 'street_name': 'Rua Manoel Antônio de Magalhães', 'neighborhood': 'Medianeira', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90880-240', 'latitude': -30.06307018, 'longitude': -51.21643935}
Address saved to database: {'street_number': '250', 'street_name': 'Avenida Presidente João Goulart', 'neighborhood': 'Centro Histórico de Porto Alegre', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90010-120', 'latitude': -30.03528083, 'longitude': -51.24316122}
Address saved to database: {'street_number': '1331', 'street_name': 'Avenida Padre Cacique', 'neighborhood': 'Praia de Belas', 'city': 'Porto Alegre', 'state': 'RS

Address saved to database: {'street_number': '1195', 'street_name': 'Rua dos Andradas', 'neighborhood': 'Centro Histórico de Porto Alegre', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '91780-010', 'latitude': -30.02987743, 'longitude': -51.22686154}
Address saved to database: {'street_number': '584', 'street_name': 'Avenida Assis Brasil', 'neighborhood': "Passo d'Areia", 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '91010-000', 'latitude': -30.00852674, 'longitude': -51.17998812}
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '409', 'street_name': 'Rua Barão do Triunfo', 'neighborhood': 'Menino Deus', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90130-101', 'latitude': -30.05256796, 'longitude': -51.21759858}
Address saved to database: 

Address saved to database: {'street_number': '14', 'street_name': 'Avenida Castelo Branco', 'neighborhood': 'Cais do Porto', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90230-210', 'latitude': -30.01438115, 'longitude': -51.21604986}
Address saved to database: {'street_number': '829', 'street_name': 'Avenida Edvaldo Pereira Paiva', 'neighborhood': 'Praia de Belas', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90110-060', 'latitude': -30.04553086, 'longitude': -51.23412735}
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '481', 'street_name': 'Rua Pinto Bandeira', 'neighborhood': 'Centro Histórico de Porto Alegre', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code

Address saved to database: {'street_number': '1144', 'street_name': 'Rua Anita Garibaldi', 'neighborhood': "Mont'Serrat", 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90450-190', 'latitude': -30.02581989, 'longitude': -51.18439658}
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '667', 'street_name': 'Rua Domingos Crescêncio', 'neighborhood': 'Santana', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90650-090', 'latitude': -30.04759556, 'longitude': -51.20358053}
Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '4248', 'street_name': 'Avenida Castelo Branco', 'neighborhood': 'Cais do Porto', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90230-210', 'latitude': -30.0167068, 'longitude': -51.21734246}
Address saved to database: {'street_number':

Address couldn't be saved to database. Data returned from reverse_geocode API: []
Address saved to database: {'street_number': '444', 'street_name': 'Rua Nunes Machado', 'neighborhood': 'Azenha', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90130-080', 'latitude': -30.05433398, 'longitude': -51.21659391}
Address saved to database: {'street_number': '421', 'street_name': 'Rua Doutor Alcídes Cruz', 'neighborhood': 'Santa Cecilia', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90630-160', 'latitude': -30.04226418, 'longitude': -51.20158588}
Address saved to database: {'street_number': '520', 'street_name': 'Rua Felizardo Furtado', 'neighborhood': 'Petrópolis', 'city': 'Porto Alegre', 'state': 'RS', 'country': 'Brazil', 'postal_code': '90670-090', 'latitude': -30.04928615, 'longitude': -51.181825}
Address saved to database: {'street_number': '85', 'street_name': 'Rua João Bonuma', 'neighborhood': 'Coronel Aparicio Borges', 'city'

## Load Phase
#### 1 - Reads data from Database and displays in current Cell.


In [1]:
from IPython.display import display, HTML
import pandas as pd
from transform_db import Database

db = Database()

with db.conn.engine.connect() as conn, conn.begin():
    data = pd.read_sql("addresses", conn)
    display(HTML(data.to_html()))



,street_number,street_name,neighborhood,city,state,country,postal_code,latitude,longitude
0,2351,Rua Coronel Aparício Borges,Coronel Aparicio Borges,Porto Alegre,RS,Brazil,90680-570,-30.066682,-51.186005
1,2136,Rua Voluntários da Pátria,Centro Histórico de Porto Alegre,Porto Alegre,RS,Brazil,90030-001,-30.014485,-51.212053
2,2098,Avenida Doutor Nilo Peçanha,Petrópolis,Porto Alegre,RS,Brazil,90470-001,-30.027792,-51.171009
3,236,Rua Mansão,Santo Antônio,Porto Alegre,RS,Brazil,90650-030,-30.053417,-51.209475
4,237,Avenida Sergipe,Glória,Porto Alegre,RS,Brazil,91720-110,-30.077903,-51.203679
5,172,Rua Humberto de Campos,Partenon,Porto Alegre,RS,Brazil,90660-280,-30.059682,-51.195864
6,167,Rua Cap Coelho Ilha Da Pintada,Arquipélago,Porto Alegre,RS,Brazil,90090-040,-30.012431,-51.258278
7,32,Avenida Alegrete,Petrópolis,Porto Alegre,RS,Brazil,90460-100,-30.039727,-51.190058
8,27,Rua Nelson Duarte Brochado,Partenon,Porto Alegre,RS,Brazil,90610-090,-30.060705,-51.177647
9,1217,Avenida Getúlio Vargas,Menino Deus,Porto Alegre,RS,Brazil,90150-005,-30.045331,-51.221243
